In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model

from sklearn.utils.class_weight import compute_class_weight

from matplotlib import pyplot as plt

from datetime import datetime

import pickle

from PIL import Image

from custom_class import *

print(f"pd.__version__: {pd.__version__}")
print(f"tf.__version__: {tf.__version__}")

image_shape = (60, 80, 3)

2025-03-12 17:52:36.061194: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 17:52:36.091935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-12 17:52:36.091960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-12 17:52:36.092688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 17:52:36.098730: I tensorflow/core/platform/cpu_feature_guar

pd.__version__: 2.2.3
tf.__version__: 2.15.1


In [2]:
speed_heads = []
angle_heads = []

In [3]:
angle_weight_paths = [
    "/mnt/c/Users/psamt/OneDrive - The University of Nottingham/_Spring/PHYS4036_Machine Learning in Science Part II/Project/auto_pilot/Models/angle_model_60x80_20250311_2005/angle_model_60x80_20250311_2005.weights.h5",
    "/mnt/c/Users/psamt/OneDrive - The University of Nottingham/_Spring/PHYS4036_Machine Learning in Science Part II/Project/auto_pilot/Models/angle_model_60x80_20250311_2013/angle_model_60x80_20250311_2013.weights.h5",
]

speed_weight_paths = [
    "/mnt/c/Users/psamt/OneDrive - The University of Nottingham/_Spring/PHYS4036_Machine Learning in Science Part II/Project/auto_pilot/Models/speed_model_60x80_20250311_2024/speed_model_20250311_2024.weights.h5",
    "/mnt/c/Users/psamt/OneDrive - The University of Nottingham/_Spring/PHYS4036_Machine Learning in Science Part II/Project/auto_pilot/Models/speed_model_60x80_20250311_2052/speed_model_60x80_20250311_2052.weights.h5",
]

In [4]:
needed_heads_angle = [
    [5, 6, 7],
    [0, 1],
]

needed_heads_speed = [
    [1, 6, 8],
    [1, 3],
]

total_num_of_heads = 10

In [5]:
mobilenetv3 = keras.applications.MobileNetV3Large(
    include_top=False,
    weights="imagenet",
    input_shape=image_shape,
    pooling=None,
    include_preprocessing=False, 
)

backbone = Model(inputs=mobilenetv3.input, outputs=mobilenetv3.get_layer("expanded_conv_14/Add").output, name="MobileNetV3Large_backbone")
neck = Model(inputs=mobilenetv3.get_layer("expanded_conv_14/Add").output, outputs=mobilenetv3.output, name="MobileNetV3Large_neck")
neck_out = neck(backbone.output)

2025-03-12 17:52:38.324806: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-12 17:52:38.350719: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-12 17:52:38.350761: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-12 17:52:38.353493: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-12 17:52:38.353529: I external/local_xla/xla/stream_executor

### Angle

#### Angle model 0

In [6]:
base_model = keras.applications.MobileNetV3Large(
    include_top=False,
    weights="imagenet",
    input_shape=image_shape,
    pooling=None,
    include_preprocessing=False, 
)

feature_extractor = base_model.output

print(feature_extractor.shape)

a_model = []

output_activation_1 = 'sigmoid'
output_activation_2 = 'linear'


# model 0
i = 0

a = feature_extractor
a = layers.Conv2D(128, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_2')(feature_extractor)
a = layers.BatchNormalization(name=f'a{i}_bn_2')(a)
a = layers.Activation('relu', name=f'a{i}_relu_2')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout_2')(a)
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 1
i += 1

a = feature_extractor
a = layers.Conv2D(496, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(feature_extractor)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_2')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_2')(a)
a = layers.Activation('relu', name=f'a{i}_relu_2')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 2
i += 1

a = feature_extractor
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])
a = layers.Conv2D(128, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)
a = layers.Dense(384, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout_2')(a)
a = layers.Dense(96, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 3
i += 1

a = feature_extractor
a = layers.Conv2D(72, (3, 3), (2, 2), padding='same', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.4, name=f'a{i}_dropout_2')(a)
a = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.1, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 4
i += 1

a = feature_extractor
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])

# Conv to reduce channels
a = layers.Conv2D(64, (3, 3), padding='same', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)

# Dense layers
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)



# model 5
i += 1

a = feature_extractor

# Conv to reduce channels
a = layers.Conv2D(64, (3, 3), padding='same', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)

# Dense layers
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 6
i += 1

a = feature_extractor
a = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)



# model 7
i += 1

a = feature_extractor
a = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(468, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(64, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 8
i += 1

a = feature_extractor
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])
a = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_attention_flatten')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(64, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)



# model 9
i += 1

a = feature_extractor
a = layers.Conv2D(86, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(168, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)

outputs_dict = {}

for i in range(len(a_model)):
    outputs_dict[f'angle{i}_output'] = a_model[i]


angle_model = Model(inputs=base_model.input, outputs=outputs_dict, name="angle_model")

angle_model.load_weights(angle_weight_paths[len(angle_heads)])

(None, 2, 3, 960)


In [7]:
needed = []
for i in needed_heads_angle[len(angle_heads)]:
    needed.append(angle_model.outputs[i])

angle_heads.append(Model(inputs=base_model.get_layer("expanded_conv_14/Add").output, outputs=needed, name=f"angle_head_{len(angle_heads)}"))

#### Angle model 1

In [8]:
base_model = keras.applications.MobileNetV3Large(
    include_top=False,
    weights="imagenet",
    input_shape=image_shape,
    pooling=None,
    include_preprocessing=False, 
)

feature_extractor = base_model.output

print(feature_extractor.shape)

a_model = []

output_activation_1 = 'sigmoid'
output_activation_2 = 'linear'


# model 0
i = 0

a = feature_extractor
a = layers.Conv2D(128, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_2')(feature_extractor)
a = layers.BatchNormalization(name=f'a{i}_bn_2')(a)
a = layers.Activation('relu', name=f'a{i}_relu_2')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout_2')(a)
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 1
i += 1

a = feature_extractor
a = layers.Conv2D(496, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(feature_extractor)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_2')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_2')(a)
a = layers.Activation('relu', name=f'a{i}_relu_2')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 2
i += 1

a = feature_extractor
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])
a = layers.Conv2D(128, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)
a = layers.Dense(384, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout_2')(a)
a = layers.Dense(96, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 3
i += 1

a = feature_extractor
a = layers.Conv2D(72, (3, 3), (2, 2), padding='same', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.4, name=f'a{i}_dropout_2')(a)
a = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.1, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 4
i += 1

a = feature_extractor
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])

# Conv to reduce channels
a = layers.Conv2D(64, (3, 3), padding='same', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)

# Dense layers
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)



# model 5
i += 1

a = feature_extractor

# Conv to reduce channels
a = layers.Conv2D(64, (3, 3), padding='same', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)

# Dense layers
a = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 6
i += 1

a = feature_extractor
a = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)



# model 7
i += 1

a = feature_extractor
a = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(468, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(64, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)


# model 8
i += 1

a = feature_extractor
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])
a = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.Flatten(name=f'a{i}_attention_flatten')(a)
a = layers.Dense(512, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(64, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)



# model 9
i += 1

a = feature_extractor
a = layers.Conv2D(86, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(a)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a_attention = layers.Conv2D(1, (1, 1), padding='same', activation="sigmoid", name=f'a{i}_attention_conv')(a)
a = layers.Multiply(name=f'a{i}_attention_apply')([a, a_attention])
a = layers.Flatten(name=f'a{i}_flatten')(a)
a = layers.Dense(168, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.3, name=f'a{i}_dropout')(a)
a = layers.Dense(1, activation=output_activation_1, name=f'a{i}_output')(a)

a_model.append(a)

outputs_dict = {}

for i in range(len(a_model)):
    outputs_dict[f'angle{i}_output'] = a_model[i]


angle_model = Model(inputs=base_model.input, outputs=outputs_dict, name="angle_model")

angle_model.load_weights(angle_weight_paths[len(angle_heads)])

(None, 2, 3, 960)


In [9]:
needed = []
for i in needed_heads_angle[len(angle_heads)]:
    needed.append(angle_model.outputs[i])

angle_heads.append(Model(inputs=base_model.get_layer("expanded_conv_14/Add").output, outputs=needed, name=f"angle_head_{len(angle_heads)}"))

#### Merge angle heads

In [10]:
angle_heads_out = [angle_head(backbone.output) for angle_head in angle_heads]
angle_heads_out

[[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'angle_head_0')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'angle_head_0')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'angle_head_0')>],
 [<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'angle_head_1')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'angle_head_1')>]]

In [11]:
# angle_head = Model(inputs=base_model.get_layer("expanded_conv_14/Add").output, outputs=angle_model.output, name="angle_head")

### Speed

#### Speed model 0

In [12]:
base_model = keras.applications.MobileNetV3Large(
    include_top=False,
    weights="imagenet",
    input_shape=image_shape,
    pooling=None,
    include_preprocessing=False, 
)

feature_extractor = base_model.output

print(feature_extractor.shape)

s_model = []

output_activation_1 = 'sigmoid'
output_activation_2 = 'linear'


# model 0
i = 0

s = feature_extractor
s = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_2')(feature_extractor)
s = layers.BatchNormalization(name=f's{i}_bn_2')(s)
s = layers.Activation('relu', name=f's{i}_relu_2')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)
s = layers.Dense(196, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 1
i += 1

s = feature_extractor
s = layers.Conv2D(256, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(feature_extractor)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_2')(s)
s = layers.BatchNormalization(name=f's{i}_bn_2')(s)
s = layers.Activation('relu', name=f's{i}_relu_2')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 2
i += 1

s = feature_extractor
s_attention = layers.Conv2D(1, (1, 1), padding='same', activation="relu", name=f's{i}_attention_conv')(s)
s = layers.Multiply(name=f's{i}_attention_apply')([s, s_attention])
s = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)
s = layers.Dense(312, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(96, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 3
i += 1

s = feature_extractor
s = layers.Conv2D(64, (3, 3), (2, 2), padding='same', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(36, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(24, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.4, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 4
i += 1

s = feature_extractor

# Conv to reduce channels
s = layers.Conv2D(64, (3, 3), padding='same', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)

# Dense layers
s = layers.Dense(52, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.4, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)



# model 5
i += 1

s = feature_extractor

# Conv to reduce channels
s = layers.Conv2D(64, (3, 3), padding='same', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)

# Dense layers
s = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 6
i += 1

s = feature_extractor
s = layers.Conv2D(72, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)



# model 7
i += 1

s = feature_extractor
s = layers.Conv2D(48, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(52, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 8
i += 1

s = feature_extractor
s_attention = layers.Conv2D(1, (1, 1), padding='same', activation="relu", name=f's{i}_attention_conv')(s)
s = layers.Multiply(name=f's{i}_attention_apply')([s, s_attention])
s = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_2')(s)
s = layers.BatchNormalization(name=f's{i}_bn_2')(s)
s = layers.Activation('relu', name=f's{i}_relu_2')(s)
s = layers.Conv2D(32, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_attention_flatten')(s)
s = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)



# model 9
i += 1

s = feature_extractor
s = layers.Conv2D(48, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s_attention = layers.Conv2D(1, (1, 1), padding='same', activation="relu", name=f's{i}_attention_conv')(s)
s = layers.Multiply(name=f's{i}_attention_apply')([s, s_attention])
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(48, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(28, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.4, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


outputs_dict = {}


for i in range(len(s_model)):
    outputs_dict[f'speed{i}_output'] = s_model[i]


speed_model = Model(inputs=base_model.input, outputs=outputs_dict, name="speed_model")

speed_model.load_weights(speed_weight_paths[len(speed_heads)])

(None, 2, 3, 960)


In [13]:
needed = []
for i in needed_heads_speed[len(speed_heads)]:
  needed.append(speed_model.outputs[i])

speed_heads.append(Model(inputs=base_model.output, outputs=needed, name=f"speed_head_{len(speed_heads)}"))
# speed_head = Model(inputs=base_model.output, outputs=speed_model.output, name="speed_head")

#### Speed model 1

In [14]:
base_model = keras.applications.MobileNetV3Large(
    include_top=False,
    weights="imagenet",
    input_shape=image_shape,
    pooling=None,
    include_preprocessing=False, 
)

feature_extractor = base_model.output

print(feature_extractor.shape)

s_model = []

output_activation_1 = 'sigmoid'
output_activation_2 = 'linear'


# model 0
i = 0

s = feature_extractor
s = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_2')(feature_extractor)
s = layers.BatchNormalization(name=f's{i}_bn_2')(s)
s = layers.Activation('relu', name=f's{i}_relu_2')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)
s = layers.Dense(196, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(128, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 1
i += 1

s = feature_extractor
s = layers.Conv2D(256, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(feature_extractor)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_2')(s)
s = layers.BatchNormalization(name=f's{i}_bn_2')(s)
s = layers.Activation('relu', name=f's{i}_relu_2')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 2
i += 1

s = feature_extractor
s_attention = layers.Conv2D(1, (1, 1), padding='same', activation="relu", name=f's{i}_attention_conv')(s)
s = layers.Multiply(name=f's{i}_attention_apply')([s, s_attention])
s = layers.Conv2D(96, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)
s = layers.Dense(312, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(96, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 3
i += 1

s = feature_extractor
s = layers.Conv2D(64, (3, 3), (2, 2), padding='same', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(36, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(24, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.4, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 4
i += 1

s = feature_extractor

# Conv to reduce channels
s = layers.Conv2D(64, (3, 3), padding='same', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)

# Dense layers
s = layers.Dense(52, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.4, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)



# model 5
i += 1

s = feature_extractor

# Conv to reduce channels
s = layers.Conv2D(64, (3, 3), padding='same', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.GlobalAveragePooling2D(name=f's{i}_GAP')(s)

# Dense layers
s = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 6
i += 1

s = feature_extractor
s = layers.Conv2D(72, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)



# model 7
i += 1

s = feature_extractor
s = layers.Conv2D(48, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(86, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(52, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


# model 8
i += 1

s = feature_extractor
s_attention = layers.Conv2D(1, (1, 1), padding='same', activation="relu", name=f's{i}_attention_conv')(s)
s = layers.Multiply(name=f's{i}_attention_apply')([s, s_attention])
s = layers.Conv2D(64, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_2')(s)
s = layers.BatchNormalization(name=f's{i}_bn_2')(s)
s = layers.Activation('relu', name=f's{i}_relu_2')(s)
s = layers.Conv2D(32, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s = layers.Flatten(name=f's{i}_attention_flatten')(s)
s = layers.Dense(32, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)



# model 9
i += 1

s = feature_extractor
s = layers.Conv2D(48, (3, 3), (2, 2), padding='same', 
                  kernel_initializer='he_uniform', name=f's{i}_conv_1')(s)
s = layers.BatchNormalization(name=f's{i}_bn_1')(s)
s = layers.Activation('relu', name=f's{i}_relu_1')(s)
s_attention = layers.Conv2D(1, (1, 1), padding='same', activation="relu", name=f's{i}_attention_conv')(s)
s = layers.Multiply(name=f's{i}_attention_apply')([s, s_attention])
s = layers.Flatten(name=f's{i}_flatten')(s)
s = layers.Dense(48, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_2')(s)
s = layers.Dropout(0.5, name=f's{i}_dropout_2')(s)
s = layers.Dense(28, activation='relu', kernel_initializer='he_uniform', name=f's{i}_dense_1')(s)
s = layers.Dropout(0.4, name=f's{i}_dropout')(s)
s = layers.Dense(1, activation=output_activation_1, name=f's{i}_output')(s)

s_model.append(s)


outputs_dict = {}


for i in range(len(s_model)):
    outputs_dict[f'speed{i}_output'] = s_model[i]


speed_model = Model(inputs=base_model.input, outputs=outputs_dict, name="speed_model")

speed_model.load_weights(speed_weight_paths[len(speed_heads)])

(None, 2, 3, 960)


In [15]:
needed = []
for i in needed_heads_speed[len(speed_heads)]:
  needed.append(speed_model.outputs[i])

speed_heads.append(Model(inputs=base_model.output, outputs=needed, name=f"speed_head_{len(speed_heads)}"))
# speed_head = Model(inputs=base_model.output, outputs=speed_model.output, name="speed_head")

#### Merge speed heads

In [16]:
speed_heads_out = [speed_head(neck_out) for speed_head in speed_heads]
speed_heads_out

[[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'speed_head_0')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'speed_head_0')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'speed_head_0')>],
 [<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'speed_head_1')>,
  <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'speed_head_1')>]]

In [17]:
# merged_model = Model(inputs=backbone.input, outputs=speed_heads_out, name="merged_model")

### Merge model

In [18]:
merged_model = Model(inputs=backbone.input, outputs=angle_heads_out + speed_heads_out, name="merged_model")
# merged_model = Model(inputs=backbone.input, outputs=[angle_head(backbone.output)] + speed_heads_out, name="merged_model")

In [19]:
# tf.keras.utils.plot_model(
#     merged_model,
#     show_shapes=True,  # Show input/output shapes
#     show_layer_names=True,  # Show layer names (e.g., "angle_hidden_2")
#     expand_nested=False,  # Keep it flat (no base_model nesting)
#     show_layer_activations=True,
#     dpi=96,  # Image resolution
# )

In [20]:
dataset_dir = "Dataset/machine-learning-in-science-ii-2025/test_data/test_data"

ds = tf.data.Dataset.list_files(
    os.path.join(dataset_dir, "*"), shuffle=False
)
image_count = len(ds)

for f in ds.take(5):
    print(f.numpy())

# Extract image IDs
def extract_image_id(file_path):
    file_path = file_path.numpy().decode("utf-8")  # Convert from bytes to string
    filename = os.path.basename(file_path)  # Extract "1.png", "10.png", etc.
    image_id = filename.split(".")[0]  # Extract "1", "10", "100"
    return image_id

image_id_list = []

for path in ds:
    image_id_list.append(extract_image_id(path))

image_id_list = np.array(image_id_list)

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def process_path(file_path):
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

def scale_ds(image):
    # Range of input image: [0, 1]
    return image * 2 - 1.0

ds = ds.map(scale_ds, num_parallel_calls=tf.data.AUTOTUNE)

def resize_ds(image):
    return tf.image.resize(image, [image_shape[0], image_shape[1]])

ds = ds.map(resize_ds, num_parallel_calls=tf.data.AUTOTUNE)

def configure_for_performance(ds):
    ds = ds.batch(1, drop_remainder=False)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

ds = configure_for_performance(ds)

b'Dataset/machine-learning-in-science-ii-2025/test_data/test_data/1.png'
b'Dataset/machine-learning-in-science-ii-2025/test_data/test_data/10.png'
b'Dataset/machine-learning-in-science-ii-2025/test_data/test_data/100.png'
b'Dataset/machine-learning-in-science-ii-2025/test_data/test_data/1000.png'
b'Dataset/machine-learning-in-science-ii-2025/test_data/test_data/1001.png'


In [21]:
# preds = merged_model.predict(ds)

In [22]:
# angle_preds = preds[0:2]
# angle_preds_list = []

# for angle_pred in angle_preds:
#     for ap in angle_pred:
#         angle_preds_list.append(ap[:, 0])

# angle_preds_list = np.array(angle_preds_list)

# angle_preds_list = np.where(np.abs(angle_preds_list - 0.5) < 0.1, angle_preds_list, (angle_preds_list - 0.5) * 1.1 + 0.5)

# angle_pred = np.mean(angle_preds_list, axis=0)
 
# print(angle_pred)

# angle_pred_df = pd.DataFrame({"image_id": image_id_list.astype(int), "angle": angle_pred}).set_index('image_id')
# # angle_pred_df.sort_index().to_csv("test_angle_pred.csv")

In [23]:
# speed_preds = preds[2:]
# speed_preds_list = []

# for speed_pred in speed_preds:
#     for sp in speed_pred:
#         speed_preds_list.append(sp[:, 0])


# # speed_preds_list = []
# # for speed_pred in speed_preds:
# #     speed_preds_list.append(speed_pred[0][:, 0])
# #     speed_preds_list.append(speed_pred[1][:, 0])

# # speed_preds = np.mean(speed_preds_list, axis=0)
# # speed_preds[0][0]

# speed_preds_list = np.array(speed_preds_list)

# speed_preds_list.mean(axis=0)

### Convert model into TFLite

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(merged_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Enable optimization
tflite_model = converter.convert()

tf_lite_file_name = f"merged_model_{image_shape[0]}x{image_shape[1]}_{total_num_of_heads}heads.tflite"

with open(tf_lite_file_name, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model: merged_model_{image_shape[0]}x{image_shape[1]}.tflite saved successfully!")

INFO:tensorflow:Assets written to: /tmp/tmpfquo_5wq/assets


INFO:tensorflow:Assets written to: /tmp/tmpfquo_5wq/assets
2025-03-12 17:53:11.555988: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-03-12 17:53:11.556044: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-03-12 17:53:11.557131: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpfquo_5wq
2025-03-12 17:53:11.588373: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-03-12 17:53:11.588407: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpfquo_5wq
2025-03-12 17:53:11.656696: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-03-12 17:53:11.691836: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-03-12 17:53:12.278928: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

TFLite model: merged_model_60x80.tflite saved successfully!


In [25]:
import numpy as np
import tensorflow.lite as tflite  # Use this if full TensorFlow is installed
# If using tflite-runtime, import this instead:
# import tflite_runtime.interpreter as tflite

# Load TFLite model
interpreter = tflite.Interpreter(model_path=tf_lite_file_name)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create a dummy input (modify based on model input shape)
input_data = "/mnt/c/Users/psamt/OneDrive - The University of Nottingham/_Spring/PHYS4036_Machine Learning in Science Part II/Project/Dataset/machine-learning-in-science-ii-2025/test_data/test_data/1.png"
input_data = np.array(Image.open(input_data).convert('RGB').resize((image_shape[1], image_shape[0])))

# convert to range(-1, 1)
input_data = input_data / 255 * 2 - 1

input_data = input_data.astype(np.float32)

input_data = np.expand_dims(input_data, axis=0)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [26]:
# output_details = interpreter.get_output_details()
# output_dict = {o['name']: o['index'] for o in output_details}

# angle_outputs = [interpreter.get_tensor(output_dict[f'StatefulPartitionedCall:{i}']) for i in range(10)]
# speed_outputs = [interpreter.get_tensor(output_dict[f'StatefulPartitionedCall:{i}']) for i in range(10, 20)]

In [27]:
output_details_sorted = sorted(output_details, key=lambda x: int(x['name'].split(':')[-1]))

all_outputs = [interpreter.get_tensor(out['index']) for out in output_details_sorted]

# Split into angle_outputs (0-9) and speed_outputs (10-19)
angle_outputs = np.array(all_outputs[:10])
speed_outputs = np.array(all_outputs[10:])

angle_outputs.mean(), speed_outputs.mean()

/tmp/ipykernel_9299/1716235872.py:9: RuntimeWarning: Mean of empty slice.
  angle_outputs.mean(), speed_outputs.mean()
/home/ubuntu/miniconda3/envs/mlis2/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(0.28291252, nan)